In [1]:
!pip install numpy

In [2]:
!pip install pandas

In [3]:
!pip install tensorflow

In [4]:
!pip install pillow


In [5]:
!pip install keras

In [6]:
!pip install opencv-python

In [7]:
!pip install tk

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the dataset
data = pd.read_csv('C:/Users/aalya/Downloads/archive (1)/fer2013.csv')

# Preprocess the dataset
def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

# Prepare the data
X_train, y_train, X_val, y_val = [], [], [], []

for index, row in data.iterrows():
    pixels = np.array(row['pixels'].split(' '), 'float32')
    pixels = pixels.reshape(48, 48, 1)  # Reshape to include channel dimension
    emotion = row['emotion']
    if row['Usage'] == 'Training':
        X_train.append(pixels)
        y_train.append(emotion)
    elif row['Usage'] == 'PublicTest':
        X_val.append(pixels)
        y_val.append(emotion)

X_train = np.array(X_train)
X_train = preprocess_input(X_train)
y_train = to_categorical(np.array(y_train), num_classes=7)

X_val = np.array(X_val)
X_val = preprocess_input(X_val)
y_val = to_categorical(np.array(y_val), num_classes=7)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Define the input shape explicitly
input_shape = (48, 48, 1)  # Assuming input images are 48x48 grayscale images

# Initialize the Sequential model
model = Sequential()

# Add the first layer with Input(shape)
model.add(Input(shape=input_shape))

# Add Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output and add Dense layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # Assuming 7 classes for emotions

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train the model using ImageDataGenerator
train_generator = datagen.flow(X_train, y_train, batch_size=64)
validation_generator = ImageDataGenerator().flow(X_val, y_val, batch_size=64)

model.fit(train_generator, epochs=50, validation_data=validation_generator)


In [ ]:
model.save('my_model.keras')

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from keras.models import load_model
from tkinter import *
from PIL import Image, ImageTk

# Load pre-trained model
model = load_model('my_model.keras')  # Path to your newly trained model
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral', 'Contempt', 'Embarrassment', 'Excitement']

# Function to detect emotion
def detect_emotion(frame):
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi_gray = roi_gray.astype("float") / 255.0
        roi_gray = tf.expand_dims(roi_gray, axis=0)
        roi_gray = tf.expand_dims(roi_gray, axis=-1)
        
        predictions = model.predict(roi_gray)
        max_index = int(np.argmax(predictions))
        predicted_emotion = emotion_labels[max_index]
        
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, predicted_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
    return frame
class Application:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.video_source = 0
        
        # Open video source (by default this will try to open the computer webcam)
        self.vid = cv2.VideoCapture(self.video_source)
        
        if not self.vid.isOpened():
            raise ValueError("Unable to open video source", self.video_source)
        
        # Create a canvas that can fit the above video source size
        self.canvas = Canvas(window, width=self.vid.get(cv2.CAP_PROP_FRAME_WIDTH), height=self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.canvas.pack()
        
        self.btn_start = Button(window, text="Start", width=50, command=self.start)
        self.btn_start.pack(anchor=CENTER, expand=True)
        
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)

        self.window.mainloop()

    def start(self):
        self.update()

    def update(self):
        ret, frame = self.vid.read()
        if ret:
            frame = detect_emotion(frame)
            self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
            self.canvas.create_image(0, 0, image=self.photo, anchor=NW)
        self.window.after(10, self.update)

    def on_closing(self):
        self.vid.release()
        self.window.destroy()

# Create and start the application
App = Application(Tk(), "Real-Time Face Emotion Detection")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━